# Short-Circuit Calculation Considering Renewable Energy Resources



In the 2016 revision of the standard, the contribution of full converter elements like wind parks of photovoltaic or wind power plants, which was previously neglected, has been integrated.  

In the paper [Short Circuit Calculation with Fullsize Converters According to IEC 60909](https://www.researchgate.net/publication/309430459_SHORT-CIRCUIT_CALCULATION_WITH_FULLSIZE_CONVERTERS_ACCORDING_TO_IEC_60909), the process is explained and an example is given. This example is used here to validate the correct implementation of the 2016 revision of the standard in pandapower.

## Example Network

The following network is introduced as example in the paper:
    
<img src="shortcircuit/example_renewables.png" width="50%">

This network can be modeled in pandapower with the line parameters given in the paper as:

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_ext_grid,
    create_line,
    create_sgen
)
from pandapower.shortcircuit import calc_sc
from pandapower.std_types import create_std_type
import numpy as np

net = create_empty_network()
for i in range(4):
    create_bus(net, vn_kv=110., index=i+1)
    
create_ext_grid(net, 1, s_sc_max_mva=20*110*np.sqrt(3), rx_max=0.1)
create_std_type(net, {"r_ohm_per_km": 0.120, "x_ohm_per_km": 0.393, "c_nf_per_km": 0.19,
                         "max_i_ka": 0.6}, "example_type")

for fb, tb, length in [(1, 2, 100), (1, 3, 50), (2, 3, 50), (3, 4, 25)]:
    create_line(net, from_bus=fb, to_bus=tb, length_km=length, std_type="example_type")

for b, p in [(2, 100), (3, 50), (4, 50)]:
    create_sgen(net, b, p_mw=p, sn_mva=p, k=1.2)

## Short Circuits Without Windparks

First, we disable the static generators that represent the wind parks and run a short circuit calculation without the wind park contribution:

In [ ]:
net.sgen.in_service = False
calc_sc(net, ip=True)

The result given in the paper without wind park contribution for a fault at bus 2 is Ikss = 2.9133 kA and ip = 5.9746 kA. Checking the results table shows that pandapower gives the same results:

In [ ]:
net.res_bus_sc

Minor deviations in the results can be explained since the values in the paper are obtained from rounded intermediate results in the paper.

## Short Circuits With Windparks

We now activate the three wind parks and run the short circuit calculation again:

In [ ]:
net.sgen.in_service = True
calc_sc(net, ip=True)

The result given in the paper with wind park contribution for a fault at bus 2 is Ikss = 3.9034 kA and ip = 7.3746 kA. Checking the results table shows that pandapower results yields the same result:

In [ ]:
net.res_bus_sc